In [1]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob


        

In [3]:
from CTImage import CTImage


ct_dataset = CTImage(patch_n=10,patch_size=64)



print(ct_dataset[0][0].shape)


torch.Size([10, 1, 64, 64])
